In [2]:
# 성능 향상이 막혔다면 다시 EDA를 해라
# 일반쓰레기, 철, 종이, 종이팩
# => 하나에 대해 분석, 수정 후 학습해보고 성능이 향상되는가 확인하기

In [3]:
from tools import *

In [4]:
cfg = EasyDict(
    target = 'General trash',
    anno_path = '../../dataset/train.json',
    output_path = './output/statistics.txt',
    classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"],
    hist_width = 1000,
    hist_height = 400
)
df,dct = load(cfg)
t_df = df[df.class_name == cfg.target]
sample(t_df, 3), df.shape, t_df.shape

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


,image_id,image_name,class_name,class_id,x_min,y_min,x_max,y_max
4407,915,train/0915.jpg,General trash,0,768.0,544.0,1023.0,748.0
20436,4278,train/4268.jpg,General trash,0,662.0,840.0,1023.0,1023.0
23182,4842,train/4832.jpg,General trash,0,248.0,169.0,959.0,907.0


(None, (23412, 8), (4206, 8))

In [5]:
hist_ann_per_img(t_df, cfg)

t_bbox_df = convert_to_bbox(t_df)
box_bbox_area(t_bbox_df, cfg)
box_aspect_ratio(t_bbox_df, cfg)

cal_statisitcs(t_df, t_bbox_df, cfg, single=True)

In [14]:
def filter(series, value, type='eq'):
    if type == 'eq': return (series == value).values
    if type == 'gt': return (series > value).values
    if type == 'get': return (series >= value).values
    if type == 'lt': return (series < value).values
    if type == 'let': return (series <= value).values
    if type == 'neq': return (series != value).values
    if type == 'in': return (series.apply(lambda x: x in value)).values

def apply_filter(df, filter):
    return df[filter]

count_upper_limit = 15
t_count_df = t_df.groupby('image_id').count().reset_index()
filter_count = filter(
    t_df.image_id,
    apply_filter(
        t_count_df, 
        filter(
            t_df.groupby('image_id').count().reset_index().iloc[:,1], 
            count_upper_limit, 
            'gt')).image_id.values,
    'in'
)
fixed_t_df = apply_filter(t_df, filter_count)
fixed_t_bbox_df = apply_filter(t_bbox_df, filter_count)

bbox_upper_limit = 333081 /1024/1024
filter_area = filter(fixed_t_bbox_df.frac_bbox_area, bbox_upper_limit, 'let')
fixed_t_df = apply_filter(fixed_t_df, filter_area)
fixed_t_bbox_df = apply_filter(fixed_t_bbox_df, filter_area)

bbox_upper_ratio = 2.82
filter_ratio = filter(fixed_t_bbox_df.aspect_ratio, bbox_upper_ratio, 'let')
fixed_t_df = apply_filter(fixed_t_df, filter_ratio)
fixed_t_bbox_df = apply_filter(fixed_t_bbox_df, filter_ratio)


In [15]:
num_problems = ['train/0402.jpg', 'train/0833.jpg', 'train/1872.jpg',
       'train/4197.jpg', 'train/4773.jpg']
area_problems = ['train/0023.jpg', 'train/0030.jpg', 'train/0058.jpg',
       'train/0079.jpg', 'train/0082.jpg', 'train/0089.jpg',
       'train/0122.jpg', 'train/0200.jpg', 'train/0221.jpg',
       'train/0224.jpg', 'train/0301.jpg', 'train/0302.jpg',
       'train/0305.jpg', 'train/0318.jpg', 'train/0332.jpg',
       'train/0355.jpg', 'train/0400.jpg', 'train/0410.jpg',
       'train/0438.jpg', 'train/0447.jpg', 'train/0471.jpg',
       'train/0481.jpg', 'train/0491.jpg', 'train/0512.jpg',
       'train/0526.jpg', 'train/0591.jpg', 'train/0623.jpg',
       'train/0630.jpg', 'train/0634.jpg', 'train/0646.jpg',
       'train/0700.jpg', 'train/0719.jpg', 'train/0754.jpg',
       'train/0812.jpg', 'train/0839.jpg', 'train/0883.jpg',
       'train/0891.jpg', 'train/0901.jpg', 'train/0932.jpg',
       'train/0941.jpg', 'train/0942.jpg', 'train/0983.jpg',
       'train/1000.jpg', 'train/1041.jpg', 'train/1076.jpg',
       'train/1089.jpg', 'train/1091.jpg', 'train/1102.jpg',
       'train/1105.jpg', 'train/1181.jpg', 'train/1184.jpg',
       'train/1205.jpg', 'train/1227.jpg', 'train/1240.jpg',
       'train/1275.jpg', 'train/1302.jpg', 'train/1377.jpg',
       'train/1411.jpg', 'train/1512.jpg', 'train/1540.jpg',
       'train/1577.jpg', 'train/1582.jpg', 'train/1609.jpg',
       'train/1667.jpg', 'train/1763.jpg', 'train/1769.jpg',
       'train/1786.jpg', 'train/1834.jpg', 'train/1861.jpg',
       'train/1888.jpg', 'train/1901.jpg', 'train/1906.jpg',
       'train/1955.jpg', 'train/1961.jpg', 'train/2041.jpg',
       'train/2054.jpg', 'train/2071.jpg', 'train/2083.jpg',
       'train/2090.jpg', 'train/2132.jpg', 'train/2135.jpg',
       'train/2149.jpg', 'train/2153.jpg', 'train/2160.jpg',
       'train/2166.jpg', 'train/2176.jpg', 'train/2194.jpg',
       'train/2202.jpg', 'train/2239.jpg', 'train/2240.jpg',
       'train/2243.jpg', 'train/2244.jpg', 'train/2251.jpg',
       'train/2321.jpg', 'train/2377.jpg', 'train/2388.jpg',
       'train/2396.jpg', 'train/2427.jpg', 'train/2508.jpg',
       'train/2509.jpg', 'train/2525.jpg', 'train/2559.jpg',
       'train/2589.jpg', 'train/2623.jpg', 'train/2663.jpg',
       'train/2672.jpg', 'train/2680.jpg', 'train/2725.jpg',
       'train/2771.jpg', 'train/2776.jpg', 'train/2808.jpg',
       'train/2841.jpg', 'train/2867.jpg', 'train/2948.jpg',
       'train/2950.jpg', 'train/2963.jpg', 'train/2989.jpg',
       'train/3012.jpg', 'train/3060.jpg', 'train/3103.jpg',
       'train/3105.jpg', 'train/3107.jpg', 'train/3201.jpg',
       'train/3206.jpg', 'train/3228.jpg', 'train/3268.jpg',
       'train/3353.jpg', 'train/3359.jpg', 'train/3388.jpg',
       'train/3392.jpg', 'train/3424.jpg', 'train/3433.jpg',
       'train/3438.jpg', 'train/3453.jpg', 'train/3463.jpg',
       'train/3466.jpg', 'train/3519.jpg', 'train/3530.jpg',
       'train/3554.jpg', 'train/3592.jpg', 'train/3593.jpg',
       'train/3594.jpg', 'train/3600.jpg', 'train/3624.jpg',
       'train/3633.jpg', 'train/3692.jpg', 'train/3703.jpg',
       'train/3707.jpg', 'train/3736.jpg', 'train/3755.jpg',
       'train/3760.jpg', 'train/3773.jpg', 'train/3788.jpg',
       'train/3832.jpg', 'train/3850.jpg', 'train/3878.jpg',
       'train/3887.jpg', 'train/3910.jpg', 'train/3912.jpg',
       'train/3925.jpg', 'train/3936.jpg', 'train/3945.jpg',
       'train/3963.jpg', 'train/3981.jpg', 'train/3988.jpg',
       'train/4005.jpg', 'train/4008.jpg', 'train/4034.jpg',
       'train/4075.jpg', 'train/4089.jpg', 'train/4150.jpg',
       'train/4208.jpg', 'train/4220.jpg', 'train/4256.jpg',
       'train/4268.jpg', 'train/4269.jpg', 'train/4287.jpg',
       'train/4312.jpg', 'train/4327.jpg', 'train/4329.jpg',
       'train/4335.jpg', 'train/4365.jpg', 'train/4384.jpg',
       'train/4459.jpg', 'train/4480.jpg', 'train/4547.jpg',
       'train/4569.jpg', 'train/4586.jpg', 'train/4614.jpg',
       'train/4628.jpg', 'train/4632.jpg', 'train/4643.jpg',
       'train/4646.jpg', 'train/4654.jpg', 'train/4657.jpg',
       'train/4683.jpg', 'train/4686.jpg', 'train/4713.jpg',
       'train/4746.jpg', 'train/4753.jpg', 'train/4781.jpg',
       'train/4815.jpg', 'train/4831.jpg', 'train/4832.jpg',
       'train/4838.jpg', 'train/4839.jpg', 'train/4857.jpg',
       'train/4880.jpg']
ratio_problems = ['train/0008.jpg', 'train/0028.jpg', 'train/0041.jpg',
       'train/0043.jpg', 'train/0054.jpg', 'train/0062.jpg',
       'train/0065.jpg', 'train/0103.jpg', 'train/0112.jpg',
       'train/0115.jpg', 'train/0131.jpg', 'train/0132.jpg',
       'train/0152.jpg', 'train/0167.jpg', 'train/0176.jpg',
       'train/0190.jpg', 'train/0224.jpg', 'train/0237.jpg',
       'train/0275.jpg', 'train/0278.jpg', 'train/0300.jpg',
       'train/0313.jpg', 'train/0344.jpg', 'train/0358.jpg',
       'train/0389.jpg', 'train/0402.jpg', 'train/0419.jpg',
       'train/0420.jpg', 'train/0600.jpg', 'train/0602.jpg',
       'train/0626.jpg', 'train/0639.jpg', 'train/0679.jpg',
       'train/0693.jpg', 'train/0694.jpg', 'train/0738.jpg',
       'train/0744.jpg', 'train/0751.jpg', 'train/0761.jpg',
       'train/0765.jpg', 'train/0804.jpg', 'train/0807.jpg',
       'train/0819.jpg', 'train/0826.jpg', 'train/0833.jpg',
       'train/0891.jpg', 'train/0910.jpg', 'train/0915.jpg',
       'train/0979.jpg', 'train/0985.jpg', 'train/1047.jpg',
       'train/1058.jpg', 'train/1063.jpg', 'train/1071.jpg',
       'train/1075.jpg', 'train/1093.jpg', 'train/1109.jpg',
       'train/1115.jpg', 'train/1131.jpg', 'train/1143.jpg',
       'train/1153.jpg', 'train/1155.jpg', 'train/1163.jpg',
       'train/1177.jpg', 'train/1190.jpg', 'train/1222.jpg',
       'train/1233.jpg', 'train/1250.jpg', 'train/1265.jpg',
       'train/1282.jpg', 'train/1292.jpg', 'train/1297.jpg',
       'train/1319.jpg', 'train/1370.jpg', 'train/1404.jpg',
       'train/1456.jpg', 'train/1474.jpg', 'train/1477.jpg',
       'train/1489.jpg', 'train/1513.jpg', 'train/1523.jpg',
       'train/1533.jpg', 'train/1548.jpg', 'train/1553.jpg',
       'train/1581.jpg', 'train/1607.jpg', 'train/1644.jpg',
       'train/1654.jpg', 'train/1701.jpg', 'train/1718.jpg',
       'train/1732.jpg', 'train/1734.jpg', 'train/1741.jpg',
       'train/1768.jpg', 'train/1769.jpg', 'train/1832.jpg',
       'train/1844.jpg', 'train/1852.jpg', 'train/1857.jpg',
       'train/1879.jpg', 'train/1896.jpg', 'train/1953.jpg',
       'train/1963.jpg', 'train/1967.jpg', 'train/1992.jpg',
       'train/2018.jpg', 'train/2025.jpg', 'train/2031.jpg',
       'train/2042.jpg', 'train/2072.jpg', 'train/2096.jpg',
       'train/2099.jpg', 'train/2100.jpg', 'train/2133.jpg',
       'train/2182.jpg', 'train/2211.jpg', 'train/2221.jpg',
       'train/2249.jpg', 'train/2260.jpg', 'train/2270.jpg',
       'train/2277.jpg', 'train/2287.jpg', 'train/2292.jpg',
       'train/2305.jpg', 'train/2315.jpg', 'train/2319.jpg',
       'train/2334.jpg', 'train/2356.jpg', 'train/2413.jpg',
       'train/2414.jpg', 'train/2423.jpg', 'train/2434.jpg',
       'train/2556.jpg', 'train/2678.jpg', 'train/2696.jpg',
       'train/2706.jpg', 'train/2741.jpg', 'train/2746.jpg',
       'train/2794.jpg', 'train/2872.jpg', 'train/2945.jpg',
       'train/2972.jpg', 'train/2983.jpg', 'train/3024.jpg',
       'train/3025.jpg', 'train/3026.jpg', 'train/3030.jpg',
       'train/3070.jpg', 'train/3100.jpg', 'train/3114.jpg',
       'train/3120.jpg', 'train/3135.jpg', 'train/3139.jpg',
       'train/3144.jpg', 'train/3152.jpg', 'train/3161.jpg',
       'train/3187.jpg', 'train/3214.jpg', 'train/3249.jpg',
       'train/3277.jpg', 'train/3281.jpg', 'train/3290.jpg',
       'train/3305.jpg', 'train/3349.jpg', 'train/3356.jpg',
       'train/3366.jpg', 'train/3383.jpg', 'train/3454.jpg',
       'train/3467.jpg', 'train/3478.jpg', 'train/3498.jpg',
       'train/3505.jpg', 'train/3507.jpg', 'train/3516.jpg',
       'train/3528.jpg', 'train/3537.jpg', 'train/3551.jpg',
       'train/3560.jpg', 'train/3566.jpg', 'train/3569.jpg',
       'train/3682.jpg', 'train/3700.jpg', 'train/3705.jpg',
       'train/3706.jpg', 'train/3722.jpg', 'train/3749.jpg',
       'train/3796.jpg', 'train/3804.jpg', 'train/3810.jpg',
       'train/3825.jpg', 'train/3836.jpg', 'train/3839.jpg',
       'train/3849.jpg', 'train/3912.jpg', 'train/3916.jpg',
       'train/3947.jpg', 'train/3954.jpg', 'train/3978.jpg',
       'train/4006.jpg', 'train/4011.jpg', 'train/4045.jpg',
       'train/4047.jpg', 'train/4062.jpg', 'train/4070.jpg',
       'train/4073.jpg', 'train/4076.jpg', 'train/4116.jpg',
       'train/4144.jpg', 'train/4148.jpg', 'train/4152.jpg',
       'train/4154.jpg', 'train/4162.jpg', 'train/4166.jpg',
       'train/4176.jpg', 'train/4190.jpg', 'train/4195.jpg',
       'train/4196.jpg', 'train/4197.jpg', 'train/4200.jpg',
       'train/4220.jpg', 'train/4227.jpg', 'train/4228.jpg',
       'train/4236.jpg', 'train/4246.jpg', 'train/4260.jpg',
       'train/4272.jpg', 'train/4280.jpg', 'train/4284.jpg',
       'train/4361.jpg', 'train/4371.jpg', 'train/4377.jpg',
       'train/4381.jpg', 'train/4387.jpg', 'train/4389.jpg',
       'train/4425.jpg', 'train/4434.jpg', 'train/4437.jpg',
       'train/4467.jpg', 'train/4468.jpg', 'train/4477.jpg',
       'train/4488.jpg', 'train/4492.jpg', 'train/4496.jpg',
       'train/4502.jpg', 'train/4505.jpg', 'train/4513.jpg',
       'train/4519.jpg', 'train/4526.jpg', 'train/4532.jpg',
       'train/4563.jpg', 'train/4571.jpg', 'train/4591.jpg',
       'train/4600.jpg', 'train/4622.jpg', 'train/4626.jpg',
       'train/4627.jpg', 'train/4653.jpg', 'train/4676.jpg',
       'train/4678.jpg', 'train/4718.jpg', 'train/4733.jpg',
       'train/4751.jpg', 'train/4767.jpg', 'train/4769.jpg',
       'train/4773.jpg', 'train/4777.jpg', 'train/4778.jpg',
       'train/4800.jpg', 'train/4816.jpg', 'train/4848.jpg',
       'train/4854.jpg']


- box 개수가 많은 것이 정말 문제일까? -> 이상치 이미지 확인해보기
- area가 작은 케이스를 잘 분류하지 못한다. -> 지우고 할 것이 아니라 이미지의 크기 키우기.
    - 몇 px부터 잘 분류하지 못하는가? -> 기준으로 적절하게 크기 키우기
- ratio에서 이상치는 어떤 box들일까? -> 이미지 확인해보기

글 흐름
1. 이상치를 제거해보았다. -> 다시 EDA를 하니 개선되어 보였다.
2. 그러나 성능은 좋지 않았다. -> 더 자세히 분석해보았다.
3. 알고보니 문제가 되는 부분은 이러이러해서 그 부분에서 접근해보았다.
4. 성능이 향상되었다. 이유를 갖고 행동하자. 끗

실제
1. 박스가 많은 것은 문제가 되지 않는다. 그러나 박스가 많은 경우 대부분 세밀하게 개체를 잡기 때문에 ratio의 분포가 굉장히 다양하다. 실제로 박스의 개수가 이상치로 분류되는 케이스의 대부분이 ratio에서도 이상치로 분류된다.
2. 학습의 안정성을 위해서라면, ratio에 제약을 거는 것이 좋아보인다. ratio가 몇 이상을 허용할 때의 성능을 기록하면서.
3. area가 작은 케이스 -> 이미지의 해상도를 높임으로써 해결한다.

# 클래스마다 갖는 ratio와 area의 분포에 차이가 있을 것

In [22]:
hist_ann_per_img(fixed_t_df, cfg)

box_bbox_area(fixed_t_bbox_df, cfg)
box_aspect_ratio(fixed_t_bbox_df, cfg)

cal_statisitcs(fixed_t_df, fixed_t_bbox_df, cfg, single=True)

In [23]:
from copy import deepcopy

fixed_dct = deepcopy(dct)
print(fixed_dct.keys())

fixed_dct['images'] = [
    x for x in fixed_dct['images'] 
    if x['id'] not in t_df.image_id.values or x['file_name'] in fixed_t_df.image_name.values]
fixed_dct['annotations'] = [
    x for x in fixed_dct['annotations'] 
    if x['image_id'] not in t_df.image_id.values or  x['image_id'] in fixed_t_df.image_id.values]
len(fixed_dct['images']),'<==',len(dct['images']), len(fixed_dct['annotations']),'<==',len(dct['annotations'])

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])


(3706, '<==', 3908, 17438, '<==', 18281)

In [14]:
save_json('output/fixed_train.json', fixed_dct)